In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from pathlib import Path
import pandas as pd

DRIVE_PATH = Path("/content/drive/MyDrive/ColabNotebooks/FinalProject/hh_vacancies_dataset.csv")

def load_dataset():
    if DRIVE_PATH.exists():
        return pd.read_csv(DRIVE_PATH)

    from google.colab import files

    uploaded = files.upload()
    if len(uploaded) == 0:
        raise RuntimeError("Файл не загружен. Повторите загрузку.")

    filename = next(iter(uploaded.keys()))
    return pd.read_csv(filename)

df = load_dataset()
print("Shape:", df.shape)
df.head()


Shape: (21709, 20)


,id,published_at,name,area_id,area_name,employer_id,employer_name,salary_from,salary_to,salary_currency,salary_gross,experience,employment,schedule,professional_role_main,professional_roles_all,key_skills,key_skills_count,description,alternate_url
0,125985240,2026-01-20T13:39:32+0300,Пикер на электрокаре,1,Москва,5756414.0,Журавлев Александр Сергеевич,7500.0,8571.0,RUR,False,noExperience,part,fullDay,Кладовщик,Кладовщик,"Электротележка, Электрокара, Пикер, Сборщик, К...",8,<p><em>Срочно требуются начинающие и опытные п...,https://hh.ru/vacancy/125985240
1,129646053,2026-01-20T12:14:51+0300,Сварщик,1,Москва,4316103.0,PROFF MONTAGE,145000.0,185000.0,RUR,False,between1And3,full,fullDay,Сварщик,Сварщик,"Сварочные работы, Сварщик, Сварка металлоконст...",10,<p>В компанию на постоянную работу требуется с...,https://hh.ru/vacancy/129646053
2,129662670,2026-01-20T15:23:34+0300,Администратор салона красоты (на запись клиентов),1,Москва,4965012.0,Грейс Солюшнс,40000.0,80000.0,RUR,False,noExperience,full,remote,Администратор,Администратор,"Телефонные переговоры, Ведение переписки, Навы...",10,<p><strong>&quot;Нежно&quot; - Московская сеть...,https://hh.ru/vacancy/129662670
3,112625062,2026-01-20T08:14:58+0300,Помощник Волан-де-Морта,1,Москва,1087973.0,ГРУППА КОМПАНИЙ ПРОСТОР,255000.0,355000.0,RUR,False,noExperience,full,fullDay,Агент по недвижимости,Агент по недвижимости,"Выявление потребностей, Грамотная устная речь,...",11,<p><strong>НЕ ЗНАЕШЬ С ЧЕГО НАЧАТЬ? НАЧНИ С ПР...,https://hh.ru/vacancy/112625062
4,129667398,2026-01-20T16:19:41+0300,Frontend-разработчик,1,Москва,3365485.0,НПЦ СпецЭлектронСистемы,150000.0,200000.0,RUR,False,between1And3,full,fullDay,"Программист, разработчик","Программист, разработчик",NaN,0,<p>В IT-отдел требуется <strong>Программист fr...,https://hh.ru/vacancy/129667398


In [4]:
print("Пропуски (доля):")
display(df.isna().mean().sort_values(ascending=False).head(15))

print("Дубликаты по id:", df["id"].duplicated().sum())


Пропуски (доля):


,0
salary_to,0.582201
key_skills,0.411995
salary_from,0.363029
salary_currency,0.305035
salary_gross,0.305035
employer_id,0.009489
published_at,0.000000
id,0.000000
area_id,0.000000
name,0.000000


Дубликаты по id: 0


In [5]:
df["id"] = df["id"].astype(str)

df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")

df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)

print("Shape after dedup:", df.shape)


Shape after dedup: (21709, 20)


In [6]:
import re
import html

TAG_RE = re.compile(r"<[^>]+>")

EMOJI_RE = re.compile(
    "["
    "\U0001F600-\U0001F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F1E0-\U0001F1FF"
    "\U00002700-\U000027BF"
    "\U0001FA70-\U0001FAFF"
    "]+",
    flags=re.UNICODE
)

def clean_text(text: str) -> str:
    if pd.isna(text):
        return ""
    text = str(text)
    text = html.unescape(text)
    text = TAG_RE.sub(" ", text)
    text = EMOJI_RE.sub(" ", text)
    text = text.replace("\xa0", " ")
    text = re.sub(r"[^0-9A-Za-zА-Яа-яЁё\s\.\,\:\;\!\?\-\+\#\/\(\)]", " ", text)
    text = re.sub(r"\s+", " ", text).strip().lower()
    return text

df["name_clean"] = df["name"].fillna("").astype(str).str.strip().str.lower()
df["description_clean"] = df["description"].apply(clean_text)
df["text_full"] = (df["name_clean"] + " " + df["description_clean"]).str.strip()
df["name_len"] = df["name_clean"].str.len()
df["desc_len"] = df["description_clean"].str.len()
df["text_len"] = df["text_full"].str.len()



In [7]:
import numpy as np
import pandas as pd

df["salary_mid"] = np.where(
    df["salary_from"].notna() & df["salary_to"].notna(),
    (df["salary_from"] + df["salary_to"]) / 2,
    np.where(df["salary_from"].notna(), df["salary_from"], df["salary_to"])
)
df.loc[df["salary_gross"] == True, "salary_mid"] *= 0.87

df_salary = df[df["salary_mid"].notna()].copy()

df_salary = df_salary[df_salary["salary_currency"].isin(["RUR", "RUB"])].copy()

df_salary = df_salary[df_salary["salary_mid"] > 0].copy()

print(df_salary.shape)



(14899, 27)


In [8]:
df = df.rename(columns={"salary_gross": "salary_was_gross"})
df_salary = df_salary.rename(columns={"salary_gross": "salary_was_gross"})

In [9]:
q = df_salary["salary_mid"].quantile([0.95, 0.99, 0.995])
print("\nКвантили salary_mid:")
display(q)


Квантили salary_mid:


,salary_mid
0.950,270000.0
0.990,500000.0
0.995,637750.0


In [10]:
df_salary.sort_values("salary_mid").head(10)[
    ["salary_mid", "salary_from", "salary_to", "salary_currency", "name", "employer_name", "alternate_url"]
]


,salary_mid,salary_from,salary_to,salary_currency,name,employer_name,alternate_url
7550,1.305,1.0,2.0,RUR,Аналитик ведущий,Ростелеком,https://hh.ru/vacancy/129797172
14397,2.000,2.0,NaN,RUR,Зубной техник керамист,Федяевский Дмитрий Викторович,https://hh.ru/vacancy/129976266
21112,50.000,50.0,NaN,RUR,Кастелянша,ФГАОУ ВО Первый МГМУ им. И.М.Сеченова Минздрав...,https://hh.ru/vacancy/130087343
18076,80.000,80.0,NaN,RUR,Офис-менеджер,Школьная Лига,https://hh.ru/vacancy/130037391
19844,200.000,200.0,NaN,RUR,Помощник менеджера проектов (маркетинговые исс...,Минькова Анна Васильевна,https://hh.ru/vacancy/130069358
11129,200.000,NaN,200.0,RUR,Frontend-разработчик (React),Morizo,https://hh.ru/vacancy/129929854
10740,250.000,250.0,NaN,RUR,SMM-менеджер в нише образования (Instagram),Yfizika,https://hh.ru/vacancy/129892107
11980,250.000,250.0,NaN,RUR,Бариста,Шаронова Ксения Борисовна,https://hh.ru/vacancy/129923164
4416,261.000,300.0,NaN,RUR,Контент-менеджер,Алексеева Дарья Николаевна,https://hh.ru/vacancy/129747177
11338,290.000,250.0,330.0,RUR,Администратор фитнес-клуба,Джим Про Реутов,https://hh.ru/vacancy/129928137


In [11]:
for thr in [10000, 20000, 30000, 40000]:
    cnt = (df_salary["salary_mid"] < thr).sum()
    share = cnt / len(df_salary)
    print(f"< {thr:>6}: {cnt:>5} вакансий ({share:.2%})")


<  10000:   490 вакансий (3.29%)
<  20000:   565 вакансий (3.79%)
<  30000:   651 вакансий (4.37%)
<  40000:   825 вакансий (5.54%)


In [12]:
df_salary.loc[df_salary["salary_mid"] < 30000, "salary_mid"].describe()

,salary_mid
count,651.000000
mean,7963.387181
std,7397.803443
min,1.305000
25%,3500.000000
50%,5000.000000
75%,9450.000000
max,29705.280000


In [13]:
HIGH_SALARY = df_salary["salary_mid"].quantile(0.99)
LOW_SALARY = 30000

print("Порог снизу:", LOW_SALARY)
print("Порог сверху (99-й перцентиль):", int(HIGH_SALARY))

before = len(df_salary)
df_salary = df_salary[df_salary["salary_mid"].between(LOW_SALARY, HIGH_SALARY)].copy()
after = len(df_salary)

print("Строк до фильтра:", before)
print("Строк после фильтра:", after)
print("Удалено:", before - after, f"({(before - after)/before:.2%})")


Порог снизу: 30000
Порог сверху (99-й перцентиль): 500000
Строк до фильтра: 14899
Строк после фильтра: 14131
Удалено: 768 (5.15%)


In [14]:
df_salary["salary_has_from"] = df_salary["salary_from"].notna().astype(int)
df_salary["salary_has_to"] = df_salary["salary_to"].notna().astype(int)


In [15]:
cat_cols = ["experience", "employment", "schedule", "professional_role_main"]

for col in cat_cols:
    df_salary[col] = df_salary[col].fillna("unknown").astype(str)


In [16]:
df_salary["key_skills_count"] = (
    pd.to_numeric(df_salary["key_skills_count"], errors="coerce")
    .fillna(0)
    .astype(int)
)


In [17]:
df_salary["pub_dow"] = df_salary["published_at"].dt.dayofweek


In [18]:
print("Итоговая форма датасета:", df_salary.shape)
df_salary.head()

Итоговая форма датасета: (14131, 30)


,id,published_at,name,area_id,area_name,employer_id,employer_name,salary_from,salary_to,salary_currency,...,name_clean,description_clean,text_full,name_len,desc_len,text_len,salary_mid,salary_has_from,salary_has_to,pub_dow
1,129646053,2026-01-20 12:14:51+03:00,Сварщик,1,Москва,4316103.0,PROFF MONTAGE,145000.0,185000.0,RUR,...,сварщик,в компанию на постоянную работу требуется свар...,сварщик в компанию на постоянную работу требуе...,7,515,523,165000.0,1,1,1
2,129662670,2026-01-20 15:23:34+03:00,Администратор салона красоты (на запись клиентов),1,Москва,4965012.0,Грейс Солюшнс,40000.0,80000.0,RUR,...,администратор салона красоты (на запись клиентов),нежно - московская сеть салонов лазерной эпиля...,администратор салона красоты (на запись клиент...,49,1116,1166,60000.0,1,1,1
3,112625062,2026-01-20 08:14:58+03:00,Помощник Волан-де-Морта,1,Москва,1087973.0,ГРУППА КОМПАНИЙ ПРОСТОР,255000.0,355000.0,RUR,...,помощник волан-де-морта,не знаешь с чего начать? начни с простора! ком...,помощник волан-де-морта не знаешь с чего начат...,23,2031,2055,305000.0,1,1,1
4,129667398,2026-01-20 16:19:41+03:00,Frontend-разработчик,1,Москва,3365485.0,НПЦ СпецЭлектронСистемы,150000.0,200000.0,RUR,...,frontend-разработчик,в it-отдел требуется программист frontend (jav...,frontend-разработчик в it-отдел требуется прог...,20,572,593,175000.0,1,1,1
5,129641722,2026-01-20 11:26:28+03:00,Специалист по документообороту,1,Москва,742469.0,ФК ЛОКОМОТИВ,NaN,120000.0,RUR,...,специалист по документообороту,обязанности: организация систематизированного ...,специалист по документообороту обязанности: ор...,30,1635,1666,104400.0,0,1,1


In [19]:
from pathlib import Path

FULL_NAME = "hh_vacancies_clean_full.parquet"
SALARY_NAME = "hh_vacancies_salary_model.parquet"

DRIVE_DIR = Path("/content/drive/MyDrive/ColabNotebooks/FinalProject/ProcessedDatasets")

def save_parquets(df_full, df_salary):
    if DRIVE_DIR.exists():
        DRIVE_DIR.mkdir(parents=True, exist_ok=True)

        full_path = DRIVE_DIR / FULL_NAME
        salary_path = DRIVE_DIR / SALARY_NAME

        df_full.to_parquet(full_path, index=False)
        df_salary.to_parquet(salary_path, index=False)

        return str(full_path), str(salary_path)

    local_dir = Path("/content")
    full_path = local_dir / FULL_NAME
    salary_path = local_dir / SALARY_NAME

    df_full.to_parquet(full_path, index=False)
    df_salary.to_parquet(salary_path, index=False)

    from google.colab import files
    files.download(str(full_path))
    files.download(str(salary_path))

    return str(full_path), str(salary_path)

save_parquets(df, df_salary)


('/content/drive/MyDrive/ColabNotebooks/FinalProject/ProcessedDatasets/hh_vacancies_clean_full.parquet',
 '/content/drive/MyDrive/ColabNotebooks/FinalProject/ProcessedDatasets/hh_vacancies_salary_model.parquet')